### **Initialize Data**

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the folder containing the normalized data extracts
data_folder = '/content/drive/MyDrive/BANA 650 - Healthcare Analytics/BANA 650 Healthcare Project/18 Datasets (Interpolation x Windows)/03. Normalized Datasets'

### **Create .npz files for model processing (72 Total)**
**Group Normalized Data by Lookback Window (5, 10 , 15, 20)**

In [ ]:
import os
import numpy as np
import pandas as pd

# Path to the folder containing normalized datasets
data_folder = '/content/drive/MyDrive/BANA 650 - Healthcare Analytics/BANA 650 Healthcare Project/18 Datasets (Interpolation x Windows)/03. Normalized Datasets'

output_folder = '/content/output'
os.makedirs(output_folder, exist_ok=True)

normalized_files = [f for f in os.listdir(data_folder) if f.startswith('normalized_labeled_sepsis') and f.endswith('.csv')]

print(f"Found {len(normalized_files)} files:")
for file in normalized_files:
    print(file)

look_back_windows = [5, 10, 15, 20]

# Function to prepare sequences
def prepare_sequences(data, look_back):
    sequences = []
    labels = []

    for hadm_id, group in data.groupby('hadm_id'):
        group = group.sort_values(by='charttime')

        # Select parameter columns for sequence preparation (exclude metadata and `sepsis_label`)
        parameter_columns = group.columns.difference(['hadm_id', 'charttime', 'sepsis_label'])

        # Extract sequences using a sliding window
        for i in range(len(group) - look_back):
            seq = group.iloc[i:i + look_back][parameter_columns].values
            label = group['sepsis_label'].iloc[i + look_back - 1]
            sequences.append(seq)
            labels.append(label)

    return np.array(sequences), np.array(labels)

# Process each file for all look-back windows
for file in normalized_files:
    print(f"Processing {file}...")

    data = pd.read_csv(os.path.join(data_folder, file))

    file_metadata = file.split('_')
    interp_level = file_metadata[-3]
    prediction_window = file_metadata[-1].replace('.csv', '')

    for look_back in look_back_windows:
        print(f"  Generating sequences for look-back window: {look_back}...")

        sequences, labels = prepare_sequences(data, look_back)

        output_path = os.path.join(output_folder, f"{file.replace('.csv', '')}_lookback_{look_back}.npz")
        np.savez(output_path, sequences=sequences, labels=labels)

        print(f"  Saved: {output_path}")


Found 18 files:
normalized_labeled_sepsis_data_interp_0_window_12.csv
normalized_labeled_sepsis_data_interp_0_window_3.csv
normalized_labeled_sepsis_data_interp_0_window_6.csv
normalized_labeled_sepsis_data_interp_1_window_12.csv
normalized_labeled_sepsis_data_interp_1_window_3.csv
normalized_labeled_sepsis_data_interp_1_window_6.csv
normalized_labeled_sepsis_data_interp_2_window_12.csv
normalized_labeled_sepsis_data_interp_2_window_3.csv
normalized_labeled_sepsis_data_interp_2_window_6.csv
normalized_labeled_sepsis_data_interp_3_window_12.csv
normalized_labeled_sepsis_data_interp_3_window_3.csv
normalized_labeled_sepsis_data_interp_3_window_6.csv
normalized_labeled_sepsis_data_interp_4_window_12.csv
normalized_labeled_sepsis_data_interp_4_window_3.csv
normalized_labeled_sepsis_data_interp_4_window_6.csv
normalized_labeled_sepsis_data_interp_5_window_12.csv
normalized_labeled_sepsis_data_interp_5_window_3.csv
normalized_labeled_sepsis_data_interp_5_window_6.csv
Processing normalized_la

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
import numpy as np
import os

# Function to create an RNN model
def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

output_folder = '/content/output'

npz_files = [f for f in os.listdir(output_folder) if f.endswith('.npz')]

results = []

interpolation_levels = range(6)  # Interpolation levels 0-5
prediction_windows = [3, 6, 12]  # Prediction times in hours
look_back_windows = [5, 10, 15, 20]  # Look-back windows in hours
outer_folds = 4

for interp_level in interpolation_levels:
    for prediction_time in prediction_windows:
        for look_back in look_back_windows:
            print(f"Processing Interpolation: {interp_level}, Prediction Time: {prediction_time} hours, Look-Back: {look_back} hours...")

            matching_files = [
                f for f in npz_files
                if f"interp_{interp_level}" in f
                and f"window_{prediction_time}" in f
                and f"lookback_{look_back}" in f
            ]

            if not matching_files:
                print(f"No files found for Interpolation: {interp_level}, Prediction Time: {prediction_time}, Look-Back: {look_back}. Skipping...")
                continue

            all_sequences = []
            all_labels = []

            for file in matching_files:
                print(f"  Loading {file}...")
                data = np.load(os.path.join(output_folder, file))
                sequences = data['sequences']
                labels = data['labels']

                all_sequences.append(sequences)
                all_labels.append(labels)

            all_sequences = np.vstack(all_sequences)
            all_labels = np.hstack(all_labels)

            all_sequences = np.nan_to_num(all_sequences, nan=0.0)

            skf = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
            outer_fold_results = []

            for fold_idx, (train_val_idx, test_idx) in enumerate(skf.split(all_sequences, all_labels)):
                print(f"  Outer Fold {fold_idx + 1}/{outer_folds}...")

                # Split into train/validation and test sets
                X_train_val, X_test = all_sequences[train_val_idx], all_sequences[test_idx]
                y_train_val, y_test = all_labels[train_val_idx], all_labels[test_idx]

                # Inner train/validation split (9/16 train, 3/16 validation)
                X_train, X_val, y_train, y_val = train_test_split(
                    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
                )

                # Apply SMOTE to the training set
                smote = SMOTE(random_state=42)
                X_train_resampled, y_train_resampled = smote.fit_resample(
                    X_train.reshape(X_train.shape[0], -1), y_train
                )
                X_train_resampled = X_train_resampled.reshape(
                    X_train_resampled.shape[0], X_train.shape[1], X_train.shape[2]
                )

                print(f"After SMOTE: X_train shape: {X_train_resampled.shape}, y_train shape: {y_train_resampled.shape}")

                model = create_rnn_model(input_shape=(X_train_resampled.shape[1], X_train_resampled.shape[2]))

                history = model.fit(
                    X_train_resampled, y_train_resampled,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=64,
                    verbose=1
                )

                test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
                print(f"  Fold {fold_idx + 1} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

                y_pred_prob = model.predict(X_test)
                auroc = roc_auc_score(y_test, y_pred_prob)
                print(f"  Fold {fold_idx + 1} - AUROC: {auroc:.4f}")

                outer_fold_results.append(auroc)

            mean_auroc = np.mean(outer_fold_results)
            std_auroc = np.std(outer_fold_results)
            print(f"Interpolation {interp_level}, Prediction Time {prediction_time}, Look-Back {look_back} - Mean AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

            results.append({
                'interpolation': interp_level,
                'prediction_time': prediction_time,
                'look_back': look_back,
                'mean_auroc': mean_auroc,
                'std_auroc': std_auroc
            })

import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv('rnn_results_stratified_kfold.csv', index=False)
print("Results saved to rnn_results_stratified_kfold.csv")

Processing Interpolation: 0, Prediction Time: 3 hours, Look-Back: 5 hours...
  Loading normalized_labeled_sepsis_data_interp_0_window_3_lookback_5.npz...
  Outer Fold 1/4...
After SMOTE: X_train shape: (4240, 5, 9), y_train shape: (4240,)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5887 - loss: 0.6831 - val_accuracy: 0.8301 - val_loss: 0.6512
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7398 - loss: 0.6582 - val_accuracy: 0.7978 - val_loss: 0.6074
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7905 - loss: 0.6064 - val_accuracy: 0.7882 - val_loss: 0.5255
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7995 - loss: 0.5099 - val_accuracy: 0.7903 - val_loss: 0.4356
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8174 - loss: 0.4365 - val_accuracy: 0.8054 - val_loss: 0.4150
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8337 - loss: 0.3937 - val_accuracy: 0.8086 - val_loss: 0.4085
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8480 - loss: 0.3715 - val_accuracy: 0.8355 - val_loss: 0.3701
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8583 - loss: 0.3462 - val_accuracy: 0.8419 - val_loss: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.6798 - loss: 0.6782 - val_accuracy: 0.7873 - val_loss: 0.6350
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7045 - loss: 0.6508 - val_accuracy: 0.7927 - val_loss: 0.5777
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7666 - loss: 0.5987 - val_accuracy: 0.7959 - val_loss: 0.4807
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7806 - loss: 0.5320 - val_accuracy: 0.7841 - val_loss: 0.4341
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7979 - loss: 0.4699 - val_accuracy: 0.7905 - val_loss: 0.4082
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8058 - loss: 0.4414 - val_accuracy: 0.8067 - val_loss: 0.3870
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8326 - loss: 0.3903 - val_accuracy: 0.8185 - val_loss: 0.3717
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8369 - loss: 0.3788 - val_accuracy: 0.8346 - val_loss: 0.

...

171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 59ms/step - accuracy: 0.6827 - loss: 0.6383 - val_accuracy: 0.9289 - val_loss: 0.1880
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.9498 - loss: 0.1877 - val_accuracy: 0.9294 - val_loss: 0.2206
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.9705 - loss: 0.1184 - val_accuracy: 0.9658 - val_loss: 0.1038
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.9817 - loss: 0.0782 - val_accuracy: 0.9837 - val_loss: 0.0558
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - accuracy: 0.9892 - loss: 0.0416 - val_accuracy: 0.9919 - val_loss: 0.0336
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.9942 - loss: 0.0244 - val_accuracy: 0.9891 - val_loss: 0.0385
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9967 - loss: 0.0175 - val_accuracy: 0.9897 - val_loss: 0.0383
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.9971 - loss: 0.0180 - val_accuracy: 0

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.6672 - loss: 0.6427 - val_accuracy: 0.9224 - val_loss: 0.2652
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9646 - loss: 0.2123 - val_accuracy: 0.9701 - val_loss: 0.1153
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 13s 55ms/step - accuracy: 0.9863 - loss: 0.0877 - val_accuracy: 0.9772 - val_loss: 0.0891
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9832 - loss: 0.0773 - val_accuracy: 0.9745 - val_loss: 0.0965
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.9893 - loss: 0.0524 - val_accuracy: 0.9832 - val_loss: 0.0691
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.9915 - loss: 0.0443 - val_accuracy: 0.9832 - val_loss: 0.0669
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.9911 - loss: 0.0426 - val_accuracy: 0.9864 - val_loss: 0.0534
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9933 - loss: 0.0325 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.6557 - loss: 0.6578 - val_accuracy: 0.9397 - val_loss: 0.2258
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9207 - loss: 0.2588 - val_accuracy: 0.9685 - val_loss: 0.0872
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.9736 - loss: 0.1195 - val_accuracy: 0.9691 - val_loss: 0.0960
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.9763 - loss: 0.1033 - val_accuracy: 0.9712 - val_loss: 0.0926
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9783 - loss: 0.0922 - val_accuracy: 0.9625 - val_loss: 0.1226
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.9819 - loss: 0.0790 - val_accuracy: 0.9718 - val_loss: 0.0939
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.9866 - loss: 0.0576 - val_accuracy: 0.9853 - val_loss: 0.0533
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.9879 - loss: 0.0486 - val_accuracy: 

### **Modeling by Lookback Window Group**

**Group the .npz files by Lookback Window**

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Path to the folder containing `.npz` files
output_folder = '/content/output'

npz_files = [f for f in os.listdir(output_folder) if f.endswith('.npz')]

look_back_groups = {}
for file in npz_files:
    look_back = int(file.split('_lookback_')[-1].replace('.npz', ''))
    if look_back not in look_back_groups:
        look_back_groups[look_back] = []
    look_back_groups[look_back].append(file)

data_by_look_back = {}

for look_back, files in look_back_groups.items():
    print(f"Preparing data for look-back {look_back}...")

    all_sequences = []
    all_labels = []

    for file in files:
        print(f"  Loading {file}...")
        data = np.load(os.path.join(output_folder, file))
        sequences = data['sequences']
        labels = data['labels']

        all_sequences.append(sequences)
        all_labels.append(labels)

    all_sequences = np.vstack(all_sequences)
    all_labels = np.hstack(all_labels)

    X_train, X_test, y_train, y_test = train_test_split(
        all_sequences, all_labels, test_size=0.2, random_state=42, stratify=all_labels
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    data_by_look_back[look_back] = (X_train, X_val, X_test, y_train, y_val, y_test)

print("Data prepared for all look-back windows.")


Preparing data for look-back 20...
  Loading normalized_labeled_sepsis_data_interp_2_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_3_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_5_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_1_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_2_window_6_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_1_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_0_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_0_window_6_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_4_window_6_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_5_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_3_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_4_window_3_lookback_20.npz...
  Loading normalized_labe

**Check that the files are prepared for modeling**

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (77469, 15, 9)
y_train shape: (77469,)
X_val shape: (19368, 15, 9)
y_val shape: (19368,)


In [ ]:
print("Training label distribution:", np.bincount(y_train.astype(int)))
print("Validation label distribution:", np.bincount(y_val.astype(int)))


Training label distribution: [ 2738 74731]
Validation label distribution: [  684 18684]


In [ ]:
print(np.isnan(X_train).any(), np.isinf(X_train).any())
print(np.isnan(y_train).any(), np.isinf(y_train).any())


True False
False False


In [ ]:
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)


In [ ]:
pip install imbalanced-learn


**Create RNN model by Lookback Window**

In [ ]:
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Function to create an RNN model
def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

output_folder = '/content/output'

# List all `.npz` files
npz_files = [f for f in os.listdir(output_folder) if f.endswith('.npz')]

look_back_groups = {}
for file in npz_files:
    look_back = int(file.split('_lookback_')[-1].replace('.npz', ''))
    if look_back not in look_back_groups:
        look_back_groups[look_back] = []
    look_back_groups[look_back].append(file)

data_by_look_back = {}

for look_back, files in look_back_groups.items():
    print(f"Preparing data for look-back {look_back}...")

    all_sequences = []
    all_labels = []

    for file in files:
        print(f"  Loading {file}...")
        data = np.load(os.path.join(output_folder, file))
        sequences = data['sequences']
        labels = data['labels']

        all_sequences.append(sequences)
        all_labels.append(labels)

    all_sequences = np.vstack(all_sequences)
    all_labels = np.hstack(all_labels)

    all_sequences = np.nan_to_num(all_sequences, nan=0.0)

    X_train, X_test, y_train, y_test = train_test_split(
        all_sequences, all_labels, test_size=0.2, random_state=42, stratify=all_labels
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(
        X_train.reshape(X_train.shape[0], -1), y_train
    )
    X_train_resampled = X_train_resampled.reshape(
        X_train_resampled.shape[0], X_train.shape[1], X_train.shape[2]
    )

    print(f"After SMOTE: X_train shape: {X_train_resampled.shape}, y_train shape: {y_train_resampled.shape}")

    data_by_look_back[look_back] = (X_train_resampled, X_val, X_test, y_train_resampled, y_val, y_test)

print("Data prepared for all look-back windows.")

for look_back, (X_train, X_val, X_test, y_train, y_val, y_test) in data_by_look_back.items():
    print(f"Training RNN for look-back window: {look_back}...")

    model = create_rnn_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=64,
        verbose=1
    )

    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Look-back {look_back} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    model.save(f"rnn_model_lookback_{look_back}_smote.h5")
    print(f"Saved model for look-back {look_back}.")


Preparing data for look-back 20...
  Loading normalized_labeled_sepsis_data_interp_2_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_3_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_5_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_1_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_2_window_6_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_1_window_12_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_0_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_0_window_6_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_4_window_6_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_5_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_3_window_3_lookback_20.npz...
  Loading normalized_labeled_sepsis_data_interp_4_window_3_lookback_20.npz...
  Loading normalized_labe

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 54s 31ms/step - accuracy: 0.8837 - loss: 0.2830 - val_accuracy: 0.9692 - val_loss: 0.1087
Epoch 2/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 53s 33ms/step - accuracy: 0.9735 - loss: 0.0900 - val_accuracy: 0.9736 - val_loss: 0.0753
Epoch 3/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 78s 31ms/step - accuracy: 0.9797 - loss: 0.0725 - val_accuracy: 0.9676 - val_loss: 0.1005
Epoch 4/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 50s 31ms/step - accuracy: 0.9833 - loss: 0.0586 - val_accuracy: 0.9771 - val_loss: 0.0672
Epoch 5/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 53s 33ms/step - accuracy: 0.9862 - loss: 0.0497 - val_accuracy: 0.9823 - val_loss: 0.0574
Epoch 6/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 49s 31ms/step - accuracy: 0.9885 - loss: 0.0411 - val_accuracy: 0.9844 - val_loss: 0.0456
Epoch 7/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 82s 31ms/step - accuracy: 0.9894 - loss: 0.0378 - val_accuracy: 0.9827 - val_loss: 0.0486
Epoch 8/10
1597/1597 ━━━━━━━━━━━━━━━━━━━━ 81s 30ms/step - accuracy: 0.9902 -

Look-back 20 - Test Loss: 0.0488, Test Accuracy: 0.9859
Saved model for look-back 20.
Training RNN for look-back window: 5...
Epoch 1/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 69s 11ms/step - accuracy: 0.8151 - loss: 0.4083 - val_accuracy: 0.8655 - val_loss: 0.2727
Epoch 2/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 62s 11ms/step - accuracy: 0.8987 - loss: 0.2481 - val_accuracy: 0.8615 - val_loss: 0.2738
Epoch 3/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 85s 11ms/step - accuracy: 0.9053 - loss: 0.2339 - val_accuracy: 0.8710 - val_loss: 0.2582
Epoch 4/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 79s 11ms/step - accuracy: 0.9115 - loss: 0.2222 - val_accuracy: 0.8819 - val_loss: 0.2443
Epoch 5/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 85s 12ms/step - accuracy: 0.9182 - loss: 0.2091 - val_accuracy: 0.8963 - val_loss: 0.2189
Epoch 6/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 79s 11ms/step - accuracy: 0.9242 - loss: 0.1988 - val_accuracy: 0.9085 - val_loss: 0.2023
Epoch 7/10
5714/5714 ━━━━━━━━━━━━━━━━━━━━ 82s 11ms/step - accuracy: 0.9307 - loss:

Look-back 5 - Test Loss: 0.2042, Test Accuracy: 0.9094
Saved model for look-back 5.
Training RNN for look-back window: 10...
Epoch 1/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 68s 17ms/step - accuracy: 0.8318 - loss: 0.3757 - val_accuracy: 0.9005 - val_loss: 0.2191
Epoch 2/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 64s 17ms/step - accuracy: 0.9299 - loss: 0.1874 - val_accuracy: 0.8908 - val_loss: 0.2535
Epoch 3/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 83s 18ms/step - accuracy: 0.9386 - loss: 0.1664 - val_accuracy: 0.9051 - val_loss: 0.2279
Epoch 4/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 64s 17ms/step - accuracy: 0.9422 - loss: 0.1576 - val_accuracy: 0.9276 - val_loss: 0.1632
Epoch 5/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 84s 18ms/step - accuracy: 0.9473 - loss: 0.1452 - val_accuracy: 0.9259 - val_loss: 0.1702
Epoch 6/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 80s 17ms/step - accuracy: 0.9507 - loss: 0.1380 - val_accuracy: 0.9332 - val_loss: 0.1622
Epoch 7/10
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 84s 18ms/step - accuracy: 0.9562 - loss: 

Look-back 10 - Test Loss: 0.1602, Test Accuracy: 0.9382
Saved model for look-back 10.
Training RNN for look-back window: 15...
Epoch 1/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 62s 25ms/step - accuracy: 0.8575 - loss: 0.3489 - val_accuracy: 0.9006 - val_loss: 0.2370
Epoch 2/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 57s 25ms/step - accuracy: 0.9516 - loss: 0.1383 - val_accuracy: 0.9304 - val_loss: 0.1679
Epoch 3/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 82s 25ms/step - accuracy: 0.9599 - loss: 0.1146 - val_accuracy: 0.9393 - val_loss: 0.1608
Epoch 4/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 82s 25ms/step - accuracy: 0.9640 - loss: 0.1017 - val_accuracy: 0.9535 - val_loss: 0.1122
Epoch 5/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 84s 26ms/step - accuracy: 0.9685 - loss: 0.0911 - val_accuracy: 0.9460 - val_loss: 0.1240
Epoch 6/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 78s 24ms/step - accuracy: 0.9713 - loss: 0.0828 - val_accuracy: 0.9563 - val_loss: 0.1053
Epoch 7/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 85s 25ms/step - accuracy: 0.9728 - loss

Look-back 15 - Test Loss: 0.0993, Test Accuracy: 0.9597
Saved model for look-back 15.


In [ ]:
test_data_by_look_back = {}

for look_back, (X_train, X_val, X_test, y_train, y_val, y_test) in data_by_look_back.items():
    test_data_by_look_back[look_back] = (X_test, y_test)


**Evaluate RNN by Lookback Window with AUROC**

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

# Function to compute AUROC and 95% CI
def compute_ci(y_test, y_pred_prob, n_bootstraps=1000, ci=95):
    auroc_values = []
    for _ in range(n_bootstraps):
        indices = resample(range(len(y_test)), replace=True)
        y_test_sample = y_test[indices]
        y_pred_sample = y_pred_prob[indices]

        try:
            auroc_values.append(roc_auc_score(y_test_sample, y_pred_sample))
        except ValueError:
            continue

    lower = np.percentile(auroc_values, (100 - ci) / 2)
    upper = np.percentile(auroc_values, 100 - (100 - ci) / 2)
    return np.mean(auroc_values), lower, upper

results = []
for look_back, (X_test, y_test) in test_data_by_look_back.items():
    print(f"Processing look-back {look_back}...")

    y_pred_prob = model.predict(X_test)

    mean_auroc, lower_ci, upper_ci = compute_ci(y_test, y_pred_prob)
    results.append((look_back, mean_auroc, lower_ci, upper_ci))
    print(f"Look-back {look_back}: AUROC = {mean_auroc:.4f}, 95% CI = [{lower_ci:.4f}, {upper_ci:.4f}]")


Processing look-back 20...
513/513 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step
Look-back 20: AUROC = 0.9977, 95% CI = [0.9971, 0.9982]
Processing look-back 5...
1964/1964 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
Look-back 5: AUROC = 0.9352, 95% CI = [0.9326, 0.9379]
Processing look-back 10...
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step
Look-back 10: AUROC = 0.9704, 95% CI = [0.9682, 0.9726]
Processing look-back 15...
757/757 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step
Look-back 15: AUROC = 0.9947, 95% CI = [0.9939, 0.9955]


### **Convert to HTML**

In [1]:
!jupyter nbconvert --to html BANA650_RNN_Modeling_and_Eval.ipynb

[NbConvertApp] Converting notebook BANA650_RNN_Modeling_and_Eval.ipynb to html
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('metadata' was unexpected)

Failed validating 'additionalProperties' in stream:

On instance['cells'][5]['outputs'][0]:
{'metadata': {'tags': None},
 'name': 'stdout',
 'output_type': 'stream',
 'text': 'Processing Interpolation: 0, Prediction Time: 3 hours, '
         'Look-Back...'}
[NbConvertApp] Writing 1356014 bytes to BANA650_RNN_Modeling_and_Eval.html
